<a href="https://colab.research.google.com/github/Jamess200/BirdnetProject/blob/main/Files/scripts/BirdNETAudio_To_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Mount Google Drive
# Mount Google Drive to access files stored in it
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Change Directory
# Change the working directory to a specific path in your Google Drive
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
#@title Set Up Repository Path & Clone Repository
import os
# Define path in Google Drive where you want to clone the repository
repo_path = '/content/drive/MyDrive/'

# Check if directory already exists, create it if not
if not os.path.exists(repo_path):
    os.makedirs(repo_path)

# Change working directory to defined path
os.chdir(repo_path)

# Clone the repository if doesnt exist, otherwise pull latest changes
if not os.path.exists(os.path.join(repo_path, 'BirdnetProject')):
    !git clone https://github.com/Jamess200/BirdnetProject
else:
    os.chdir('BirdnetProject')
    !git pull

# Verify cloned repository
!ls

remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 22 (delta 11), reused 1 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (22/22), 689.33 KiB | 281.00 KiB/s, done.
From https://github.com/Jamess200/BirdnetProject
   4e10576..c54763d  main       -> origin/main
Updating 1e003d5..c54763d
error: Your local changes to the following files would be overwritten by merge:
	Files/data/CSV_data/XC/XC_25_Merged.xlsx
Please commit your changes or stash them before you merge.
Aborting
Files  Lost_Data_Prediction.ipynb  Main_AI_Human.ipynb	README.md


In [ ]:
#@title Install Required Python Packages
# Install required packages
!pip3 install birdnetlib
!pip3 install tflite-runtime
!pip3 install resampy
!pip3 install ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 60.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=8a20dc3a9e5f17956597059c776c92549d76e368cdd75b31a5ebcdfb1492eaf1
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg


In [ ]:
#@title Import Necessary Libraries
# Import necessary libraries
import resampy
import birdnetlib
import tflite_runtime
import ffmpeg
import pandas as pd
from birdnetlib import Recording
from birdnetlib.analyzer import Analyzer
from datetime import datetime

In [ ]:
#@title Set Up Directory and File Paths
# Define the directory where the data is stored
dataDir = '/content/drive/MyDrive/HAUtrans/Trans5/'

# List all files in the directory
file_names = os.listdir(dataDir)

# Print the file names to verify the data files
print(file_names)

['ED3_20240502_040000.wav', 'ED3_20240502_050000.wav', 'ED3_20240502_060000.wav', 'ED3_20240502_070000.wav', 'ED3_20240502_080000.wav', 'ED3_20240502_090000.wav', 'ED3_20240503_040000.wav', 'ED3_20240503_050000.wav', 'ED3_20240503_060000.wav', 'ED3_20240503_070000.wav', 'ED3_20240503_080000.wav', 'ED3_20240503_090000.wav', 'ED3_20240504_040000.wav', 'ED3_20240504_050000.wav', 'ED3_20240504_060000.wav', 'ED3_20240504_070000.wav', 'ED3_20240504_080000.wav', 'ED3_20240504_090000.wav', 'ED3_20240505_040000.wav', 'ED3_20240505_050000.wav', 'ED3_20240505_060000.wav', 'ED3_20240505_070000.wav', 'ED3_20240505_080000.wav', 'ED3_20240505_090000.wav', 'ED3_20240506_040000.wav', 'ED3_20240506_050000.wav', 'ED3_20240506_060000.wav', 'ED3_20240506_070000.wav', 'ED3_20240506_080000.wav', 'ED3_20240506_090000.wav', 'ED3_20240507_040000.wav', 'ED3_20240507_050000.wav', 'ED3_20240507_060000.wav', 'ED3_20240507_070000.wav', 'ED3_20240507_080000.wav', 'ED3_20240507_090000.wav', 'ED3_20240508_040000.wav', 

In [ ]:
#@title Initialise BirdNET Analyzer
# Initialise the BirdNET-Analyzer model
analyzer = Analyzer()
# Define the coordinates and date for the recordings
testamp = [52.911,-2.4441] # near Market Drayton year=2024, month=4, day=20

Labels loaded.
load model True
Model loaded.
Labels loaded.
load_species_list_model
Meta model loaded.


In [ ]:
#@title Analyse Audio Files and Store Results
# Create a dictionary to store the results
results = {}
# Loop through all file names
for file_name in file_names:
    try:
      # Include the correct path to the data files
      file_path = os.path.join(dataDir, file_name)
      recording = Recording(
          analyzer,
          file_path,  # path to the recording file
          lat=testamp[0], # latitude of the recording location
          lon=testamp[1], # longitude of the recording location
          date=datetime(year=2024, month=4, day=20), # use date or week_48
          min_conf=0.25,
      )
      # Analyse the recording and store the result in the dictionary
      recording.analyze()
      results[file_name] = recording.detections
    except Exception as e:
      print(f"Error processing {file_name}: {e}")

read_audio_data
read_audio_data: complete, read  600 chunks.
analyze_recording ED3_20240502_040000.wav
recording has lon/lat
set_predicted_species_list_from_position
return_predicted_species_list
15
145 species loaded.


KeyboardInterrupt: 

In [ ]:
#@title Convert Results to DataFrame and Save as CSV
# Flatten the results dictionary
flattened_data = []
for filename, records in results.items():
    for record in records:
        record['filename'] = filename
        flattened_data.append(record)

# Convert the flattened data to a DataFrame
df = pd.DataFrame(flattened_data)

# Save the DataFrame to a CSV file
csv_file_path = '/content/drive/MyDrive/BirdnetProject/Files/data/CSV_data/transect_5.csv'
df.to_csv(csv_file_path, index=False)

# Confirm that the results have been saved successfully
print(f"Results have been successfully converted to CSV and saved to {csv_file_path}")

In [ ]:
#@title Load and Verify CSV Data
# Load the saved CSV file into a DataFrame to verify the contents
csv_file_path = '/content/drive/MyDrive/BirdnetProject/Files/data/CSV_data/transect_5.csv'
df = pd.read_csv(csv_file_path)

# Print the DataFrame to verify the data
print(df.head)

In [ ]:
#@title Change Working Directory
# Change working directory to defined path
os.chdir('/content/drive/MyDrive/BirdnetProject/Files/data/CSV_data')

# Verify working directory
print(os.getcwd())

In [ ]:
#@title Create Merged Dataframe File For All Transects
import pandas as pd

# Read in the CSV files
df5 = pd.read_csv('transect_5.csv')
df6 = pd.read_csv('transect_6.csv')
df7 = pd.read_csv('transect_7.csv')
df8 = pd.read_csv('transect_8.csv')

# Add a column to each dataframe to indicate the transect source
df5['transect'] = 5
df6['transect'] = 6
df7['transect'] = 7
df8['transect'] = 8

# Function to extract date and hour from the filename
def extract_datetime(filename):
    date_str = filename.split('_')[1]
    time_str = filename.split('_')[2].split('.')[0]
    datetime_str = date_str + ' ' + time_str
    datetime = pd.to_datetime(datetime_str, format='%Y%m%d %H%M%S')
    return datetime

# Apply the function to create a new datetime column
df5['datetime'] = df5['filename'].apply(extract_datetime)
df6['datetime'] = df6['filename'].apply(extract_datetime)
df7['datetime'] = df7['filename'].apply(extract_datetime)
df8['datetime'] = df8['filename'].apply(extract_datetime)

# Function to convert seconds to hour:minutes:seconds format based on the datetime
def seconds_to_hms(datetime, seconds):
    new_time = datetime + pd.to_timedelta(seconds, unit='s')
    return new_time.strftime('%-H:%M:%S')

# Apply the function to convert start_time and end_time
df5['start_time'] = df5.apply(lambda row: seconds_to_hms(row['datetime'], row['start_time']), axis=1)
df5['end_time'] = df5.apply(lambda row: seconds_to_hms(row['datetime'], row['end_time']), axis=1)
df6['start_time'] = df6.apply(lambda row: seconds_to_hms(row['datetime'], row['start_time']), axis=1)
df6['end_time'] = df6.apply(lambda row: seconds_to_hms(row['datetime'], row['end_time']), axis=1)
df7['start_time'] = df7.apply(lambda row: seconds_to_hms(row['datetime'], row['start_time']), axis=1)
df7['end_time'] = df7.apply(lambda row: seconds_to_hms(row['datetime'], row['end_time']), axis=1)
df8['start_time'] = df8.apply(lambda row: seconds_to_hms(row['datetime'], row['start_time']), axis=1)
df8['end_time'] = df8.apply(lambda row: seconds_to_hms(row['datetime'], row['end_time']), axis=1)

# Concatenate the dataframes
concatenated_df = pd.concat([df5, df6, df7, df8], ignore_index=True)

# Extract date and start hour
concatenated_df['date'] = concatenated_df['datetime'].dt.date
concatenated_df['start_hour'] = concatenated_df['datetime'].dt.hour

# Sort the dataframe by transect, date, start_time, common_name, and confidence
concatenated_df = concatenated_df.sort_values(by=['transect', 'date', 'start_time', 'common_name', 'confidence'])

# Reorder the columns to have transect, date, start_hour, time, and readable start/end times
reordered_columns = ['transect', 'date', 'start_hour', 'start_time', 'end_time', 'common_name', 'scientific_name', 'confidence', 'label', 'filename']
concatenated_df = concatenated_df[reordered_columns]

# Display the first few rows of the concatenated dataframe
print("\nNew Dataframe:")
print(concatenated_df.head())
print(concatenated_df.tail())


In [ ]:
#@title Save data as a CSV File
# Save the concatenated dataframe to a CSV file
output_filename = 'transect_combined.csv'
concatenated_df.to_csv(output_filename, index=False)
print(f"The combined dataframe has been saved to {output_filename}")

In [ ]:
#@title Save Data as a Excel File
# Save the dataframe to an Excel file
output_file = 'Bird_Detection_Data.xlsx'
concatenated_df.to_excel(output_file, index=False)
print(f"\nDataframe has been saved to {output_file}")